In [21]:
import os

import pymc as pm
import pandas as pd
import numpy as np
import pickle
import cloudpickle
import gzip
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import cobra
sns.set(context="talk", style="ticks", color_codes=True, rc={"legend.frameon": False})

%matplotlib inline

In [22]:
os.getcwd()

'/Users/mahs128/Repos/syn_bmca/src'

In [24]:
syn_model_fname = "../models/iJB785_no_zero_flux_flipped.xml"
syn_model =cobra.io.read_sbml_model(syn_model_fname)


Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


In [25]:
with gzip.open("Syn_80k.pgz", "rb") as f:
    advi_results = cloudpickle.load(f)

In [26]:
with gzip.open("Syn_pymcmodel_data.pgz", "rb") as f:
    pymc_data = cloudpickle.load(f)

In [27]:
pymc_data.keys()

dict_keys(['model', 'en', 'xn', 'x_inds', 'e_inds', 'll', 'v_star'])

In [28]:
dir(advi_results['model'])

['Ex',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_config_context',
 '_context_class',
 '_coords',
 '_dim_lengths',
 '_parent',
 '_pytensor_config',
 '_repr_latex_',
 '_validate_name',
 'add_coord',
 'add_coords',
 'add_named_variable',
 'basic_RVs',
 'check_bounds',
 'check_start_vals',
 'chi_obs',
 'chi_ss',
 'compile_d2logp',
 'compile_dlogp',
 'compile_fn',
 'compile_logp',
 'contexts',
 'continuous_value_vars',
 'coords',
 'create_value_var',
 'd2logp',
 'datalogp',
 'debug',
 'deterministics',
 'dim_lengths',
 'discrete_value_vars',
 'dlogp',
 'eval_rv_shapes',
 'e

In [29]:
advi_results['model'].model

/Users/mahs128/.pyenv/versions/3.11.5/envs/bmca/lib/python3.11/site-packages/pymc/model/core.py:568: FutureWarning: Model.model property is deprecated. Just use Model.
  warnings.warn("Model.model property is deprecated. Just use Model.", FutureWarning)


 ex_kinetic_entries ~ HalfNormal(0, 1)
ex_capacity_entries ~ Laplace(0, 0.01)
     log_e_measured ~ Normal(<constant>, 0.2)
   log_e_unmeasured ~ Laplace(0, 0.1)
               yn_t ~ Normal(0, 10)
                 Ex ~ Deterministic(f(ex_capacity_entries, ex_kinetic_entries))
           log_en_t ~ Deterministic(f(log_e_unmeasured, log_e_measured))
             chi_ss ~ Deterministic(f(yn_t, log_e_unmeasured, log_e_measured, ex_capacity_entries, ex_kinetic_entries))
            chi_obs ~ Normal(f(yn_t, log_e_unmeasured, log_e_measured, ex_capacity_entries, ex_kinetic_entries), 0.2)

In [38]:
trace = advi_results['hist'].sample(200)

In [34]:
trace.posterior['Ex']

<xarray.DataArray 'Ex' (chain: 1, draw: 100, Ex_dim_0: 570, Ex_dim_1: 769)>
array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 1.30119029e-02, -9.75201993e-04,  6.30601296e-03, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 1.89791700e-02, -1.18514075e-02,  1.66969779e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [-2.73606800e-02,  2.52711408e-02,  1.06414587e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  1.42422832e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00, -2.39634235e+00]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-4.13434244e-03, -1.03166537e-02, -1.44559433e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.29384764e-02,  1.01229881e-02,  9.76629914e-04, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
...
         [-8.15741376e-03,  4.53071902e-03,  2.32671008e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  1.92833930e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00, -4.64446684e-01]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 3.22265281e-03, -1.26339533e-02,  1.24686954e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 2.15413445e-03,  1.13637723e-02,  6.10895289e-03, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [-1.32577946e-03, -1.31281756e-02,  1.77540496e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  1.30054409e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00, -1.06351593e+00]]]])
Coordinates:
  * chain     (chain) int64 0
  * draw      (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98 99
  * Ex_dim_0  (Ex_dim_0) int64 0 1 2 3 4 5 6 7 ... 563 564 565 566 567 568 569
  * Ex_dim_1  (Ex_dim_1) int64 0 1 2 3 4 5 6 7 ... 762 763 764 765 766 767 768

## Make FCC box plots

In [36]:
ll = pymc_data['ll']
ll

In [ ]:
r_labels = [r.id for r in syn_model.reactions]
target_metab = 'EX_sucr_e'

In [44]:


fcc = pd.DataFrame(
    np.array(
        [
            ll.flux_control_coefficient(Ex=ex)[r_labels.index(target_metab)]
            for ex in tqdm(trace.posterior['Ex'][0].to_numpy())
        ]
    ),
    columns=r_labels,
)

In [ ]:
fcc_sort = fcc.reindex(columns=fcc.median().sort_values().index)
# fcc_prior_sort = fcc_prior.reindex(columns=fcc.median().sort_values().index)


# Calculate the fcc values that have 95% Highest Posterior Densities with a consistent direction.
hpd = pm.hpd(fcc_sort.values)
# hpd_prior = pm.hpd(fcc_prior_sort.values)

fcc_consistent = np.sign(hpd[:, 0]) == np.sign(hpd[:, 1])